In [3]:
import pickle
import pandas as pd
import math
import json

In [4]:
data = json.load(open('video_info_copy.json'))

File is structured with master keys being name of file, the values being other dictionaries, with inner keys being the frame number, and the value being an array of border coordinates and probability

Step 1: loop through dictionaries and make dataframe

In [5]:
data['/home/cusp/cmydlarz/redhook_video/data/1573276672.ts']

{'308': [[[18.5, 321.0, 191, 132]], [0.225791335105896]],
 '310': [[[18.5, 320.5, 193, 133]], [0.2184540182352066]],
 '630': [[[19.0, 316.5, 192, 139], [27.0, 340.5, 176, 115]],
  [0.5922264456748962, 0.5686070322990417]],
 '632': [[[12.5, 330.0, 203, 132], [24.5, 341.0, 181, 110]],
  [0.7574986815452576, 0.7172761559486389]]}

In [8]:
data['/home/cusp/cmydlarz/redhook_video/data/1573276672.ts']['308'][0][0][2] * data['/home/cusp/cmydlarz/redhook_video/data/1573276672.ts']['308'][0][0][3]

25212

In [10]:
start_timestamp = []
frame = []
actual_timestamp = []
area = []
probability = []

In [11]:
for master_key in data:
    for key in data[master_key]:
        start_timestamp.append(int(master_key[39:49]))
        frame.append(int(key))
        actual_timestamp.append(int(int(master_key[39:49]) + int(key)/15))
        area.append(data[master_key][key][0][0][2] * data[master_key][key][0][0][3])
        probability.append(data[master_key][key][1][0])

In [12]:
df = pd.DataFrame(data = {'start_timestamp': start_timestamp, 'frame': frame, \
                          'actual_timestamp': actual_timestamp, 'area': area, \
                          'probability': probability})

In [13]:
df.head()

,start_timestamp,frame,actual_timestamp,area,probability
0,1573276672,308,1573276692,25212,0.225791
1,1573276672,310,1573276692,25669,0.218454
2,1573276672,630,1573276714,26688,0.592226
3,1573276672,632,1573276714,26796,0.757499
4,1573061868,164,1573061878,25134,0.202466


In [23]:
len(df)

103710

In [14]:
def get_probability_locs(df, column, value, locs_list):
    for i in df[column].index:
        #print(i)
        if(math.isclose(df[column][i], value, rel_tol=1e-5) and (i not in locs_list)):
            locs_list.append(i)

In [15]:
max_set = []

In [16]:
def truck_filter(df, max_set, window_size, min_area, min_probability):
    """
    Returns an array of indices that most likely correspond to trucks, based on the probability value given by
    the image recognition algorithm.
    
    Parameters
    ----------
    df : dataframe
        The dataframe containing information gathered from an image recognition algorithm. Should contain 
        a starting timestamp column, a frame column, an area column, and a probability column.
        
    max_set : array
        The array to store the indices of likely instances of a truck.
        
    window_size : int
        The number of seconds before and after the timestamp of each row.
        
    min_area : int
        The minimum area of the bounding box. 
    
    min_probability : float
        The minimum probability given by the classifier.
        
    """
    for i in df.index:
        #Gets subsection of dataframe inside the window 
        min_timestamp = int(df['actual_timestamp'][i] - window_size/2)
        print(min_timestamp)
        max_timestamp = int(df['actual_timestamp'][i] + window_size/2)
        print(max_timestamp)
        window_df = df[(min_timestamp <= df['actual_timestamp']) & (max_timestamp >= df['actual_timestamp'])]
        #print(window_df)

        #Checks conditions for probability, window size, etc
        if window_df['probability'].max() >= min_probability and window_df['area'].max() >= min_area:
            #print('true')
            get_probability_locs(window_df, 'probability', window_df['probability'].max(), max_set)

In [17]:
truck_filter(df, max_set, 10, 80000, 0.7)

In [18]:
len(max_set)

7615

In [19]:
max_dataframe = df.iloc[max_set]

In [20]:
max_dataframe.head()

,start_timestamp,frame,actual_timestamp,area,probability
10,1573061868,648,1573061911,276490,0.988503
16,1573061868,660,1573061912,153846,0.974143
20,1573061868,722,1573061916,119448,0.849568
32,1573830164,72,1573830168,207648,0.995947
45,1573830164,1518,1573830265,294148,0.969995


In [21]:
max_dataframe.to_csv('max_dataframe.csv')

In [5]:
1573061910 - 1573061868

42